# Concept
predict how much revenue a given video idea will produce for tiger fitness. we also want to predict views, percent viewed, and change in subscribers

# Features (input 1-7, output 8-14)
1. video duration (bucketed, analytics based)
2. title polarity
3. title length (characters)
4. n links in description
5. category of video (vlog, training, nutrition, personal story, supplements and steroids) 
6. product price range (bucketed, intuition based)
7. product type (apparel, supplements, food, coaching)
8. sales made
9. views 
10. subs 
11. average percent viewed
12. likes
13. Impressions
14. Impression click through

# Models
1. revenue prediction (predict sales volume, multiply by price range to give a revenue range 
2. views prediction
3. change in subscribers prediction
4. average percent viewed prediction
5. likes prediction

Data Sources...

manual entry or unsupervised learning: 5

youtube analytics report - video analytics overview: 2, 3, 9, 10, 11, 12

youtube analytics report - video analytics engadgement: 8, 13, 14